In [134]:
from bs4 import BeautifulSoup
import numpy as np
import requests

In [135]:
url="https://docs.aws.amazon.com/cli/latest/reference/ec2/index.html"

res=requests.get(url)

soup=BeautifulSoup(res.content,"lxml")

In [136]:
cmd_pages={}
cmd_html=soup.find(id="available-commands").find_all("li")
for commands in cmd_html:
        link=commands.find("a")["href"]
        link="https://docs.aws.amazon.com/cli/latest/reference/ec2/"+link
        cmd_pages[commands.text]=link

In [137]:
# cmd_pages

In [145]:
cmd_aim=[]
cmd_description=[]
cmd_code=[]
commands=[]

for command,link in list(cmd_pages.items())[:200]:
        print(link)
        res=requests.get(link)
        soup=BeautifulSoup(res.content,"lxml")

        if soup.find(id="examples"):
                commands.append(command)
                example_html=soup.find(id="examples")
                cmd_aim.append(example_html.find_all("p")[0].text)
                cmd_description.append(example_html.find_all("p")[1].text)
                cmd_code.append(example_html.find("div",class_="highlight-python").text)
                

https://docs.aws.amazon.com/cli/latest/reference/ec2/accept-reserved-instances-exchange-quote.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/accept-transit-gateway-multicast-domain-associations.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/accept-transit-gateway-peering-attachment.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/accept-transit-gateway-vpc-attachment.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/accept-vpc-endpoint-connections.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/accept-vpc-peering-connection.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/advertise-byoip-cidr.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/allocate-address.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/allocate-hosts.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/allocate-ipam-pool-cidr.html
https://docs.aws.amazon.com/cli/latest/reference/ec2/apply-security-groups-to-client-vpn-target-network.htm

In [146]:
cmd_code[:5]

['aws ec2 accept-reserved-instances-exchange-quote --reserved-instance-ids 7b8750c3-397e-4da4-bbcb-a45ebexample --target-configurations OfferingId=b747b472-423c-48f3-8cee-679bcexample\n\n',
 'aws ec2 accept-transit-gateway-peering-attachment \\\n    --transit-gateway-attachment-id tgw-attach-4455667788aabbccd \\\n    --region us-east-2\n\n',
 'aws ec2 accept-transit-gateway-vpc-attachment \\\n    --transit-gateway-attachment-id tgw-attach-0a34fe6b4fEXAMPLE\n\n',
 'aws ec2 accept-vpc-endpoint-connections --service-id vpce-svc-03d5ebb7d9579a2b3 --vpc-endpoint-ids vpce-0c1308d7312217abc\n\n',
 'aws ec2 accept-vpc-peering-connection --vpc-peering-connection-id pcx-1a2b3c4d\n\n']

In [147]:
import pandas as pd

df=pd.DataFrame({"Command Name":commands,"Aim":cmd_aim,"Description":cmd_description,"code":cmd_code})


In [158]:
import re
def preprocessing(text):
    text=text.lower()
    text=re.sub("example 1:","",text)
    return text

def preprocess_code(text):
    # text=re.sub("\\\n","",text)
    text=re.sub(r"\n\n","",text)
    text=re.sub(r"\\","",text)
    return text

In [159]:
df["Aim"]=df["Aim"].apply(preprocessing)
df["Description"]=df["Description"].apply(preprocessing)
df["code"]=df["code"].apply(preprocess_code)

In [154]:
df

,Command Name,Aim,Description,code
0,accept-reserved-instances-exchange-quote,to perform a convertible reserved instance exc...,this example performs an exchange of the speci...,aws ec2 accept-reserved-instances-exchange-quo...
1,accept-transit-gateway-peering-attachment,to accept a transit gateway peering attachment,the following accept-transit-gateway-peering-a...,aws ec2 accept-transit-gateway-peering-attachm...
2,accept-transit-gateway-vpc-attachment,to accept a request to attach a vpc to a trans...,the following accept-transit-gateway-vpc-attac...,aws ec2 accept-transit-gateway-vpc-attachment ...
3,accept-vpc-endpoint-connections,to accept an interface endpoint connection req...,this example accepts the specified endpoint co...,aws ec2 accept-vpc-endpoint-connections --serv...
4,accept-vpc-peering-connection,to accept a vpc peering connection,this example accepts the specified vpc peering...,aws ec2 accept-vpc-peering-connection --vpc-pe...
...,...,...,...,...
184,describe-carrier-gateways,to describe all carrier gateways,the following describe-carrier-gateways exampl...,aws ec2 describe-carrier-gateways
185,describe-classic-link-instances,to describe linked ec2-classic instances,this example lists all of your linked ec2-clas...,aws ec2 describe-classic-link-instances
186,describe-client-vpn-authorization-rules,to describe the authorization rules for a clie...,the following describe-client-vpn-authorizatio...,aws ec2 describe-client-vpn-authorization-rule...
187,describe-client-vpn-connections,to describe the connections to a client vpn en...,the following describe-client-vpn-connections ...,aws ec2 describe-client-vpn-connections \\n ...


In [160]:
df.to_csv("Scrapped_AWS_CLI_commands.csv")